# Quick Start

In this notebook, we will help you be familiar with Evox through an simple example.

To start with, make sure you have already installed the EvoX.

In [30]:
# install evox, skip it if you have already installed evox
try:
    import evox
except ImportError:
    !pip install --disable-pip-version-check --upgrade -q evox
    import evox

In [31]:
# the dependent packages or functions in this example
from typing import List

import torch

from evox.algorithms import PSO
# from evox.problems import Ackley
from evox.workflows import EvalMonitor, StdWorkflow
from evox.core import Problem

## Create an algorithm and a problem

EvoX offers a comprehensive suite of 50+ Evolutionary Algorithms (EAs) and a wide range of 100+ Benchmark Problems/Environments, all benefiting from distributed GPU-acceleration. 

Here we will create a PSO algorithm and an Ackley function.

For more detailed list, please refer to our API documentation. [List of Algorithms](https://evox.readthedocs.io/en/latest/api/algorithms/index.html) and [List of Problems](https://evox.readthedocs.io/en/latest/api/problems/index.html).

In [32]:
class Sphere(Problem):
    def __init__(self):
        super().__init__()

    def evaluate(self, pop: torch.Tensor):
        return (pop**2).sum(-1)
    
algorithm = PSO(
    pop_size=100,
    lb=torch.tensor([-10.0]),
    ub=torch.tensor([10.0]),
    w=0.6,
    phi_p=2.5,
    phi_g=0.8,
    )
problem = Sphere()

## Create an workflow

Now we want to run the algorithm and solve the problem.

To accomplish this, we need to create a "**workflow**" of the `Workflow` class.

A workflow outlines the series of steps required to accomplish a task or project. In EvoX, a workflow represents the overall process of evolutionary computation. 

In [33]:
workflow = StdWorkflow()
workflow.setup(
    algorithm=algorithm,
    problem=problem,
    )

Since we adopt the functional programming paradigm. We must explicitly initialize and use the state of a module.

In [34]:
for _ in range(100):
    workflow.step()

Notice that we are passing state as an argument of step and it returns a new state. This is exactly how the stateful computation works in functional programming.

And you may also notice that the step doesn’t give any feedback, like the result of the optimization. This is because we are missing another component in our workflow. Introducing monitor.

## Set an Monitor

Monitor is a standard class in EvoX to monitor the intermediate values inside a optimization process. Information like fitness or population can be easily obtained by the monitor.

Now, create a “Evalution monitor”:

In [35]:
monitor = EvalMonitor()

The monitor can be plugged into the workflow.

In [36]:
workflow.setup(
    algorithm=algorithm,
    problem=problem,
    monitor=monitor,
    )

Now, re-initialize the workflow, and executed it again.

In [37]:
for _ in range(10):
    workflow.step()
    print(workflow.get_submodule("monitor").topk_fitness.float())

tensor([0.0008])
tensor([0.0008])
tensor([0.0008])
tensor([0.0005])
tensor([0.0005])
tensor([0.0005])
tensor([2.3342e-05])
tensor([2.3342e-05])
tensor([1.6017e-05])
tensor([1.3345e-05])
